In [54]:
import numpy as np
import pandas as pd
import warnings 
import pandas as pd
from IPython.display import display

# Show all columns and full width
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
warnings.filterwarnings('ignore')

# --- Helper feature engineering functions ---

def create_date_features(df):
    df['month'] = df.date.dt.month.astype('int8')
    df['day_of_month'] = df.date.dt.day.astype('int8')
    df['day_of_year'] = df.date.dt.dayofyear.astype('int16')
    df['week_of_month'] = ((df.date.dt.day - 1) // 7 + 1).astype('int8')
    df['week_of_year'] = df.date.dt.isocalendar().week.astype('int8')
    df['day_of_week'] = (df.date.dt.dayofweek + 1).astype('int8')
    df['year'] = df.date.dt.year.astype('int32')
    df['is_wknd'] = (df.date.dt.weekday // 4).astype('int8')
    df['quarter'] = df.date.dt.quarter.astype('int8')
    df['is_month_start'] = df.date.dt.is_month_start.astype('int8')
    df['is_month_end'] = df.date.dt.is_month_end.astype('int8')
    df['is_quarter_start'] = df.date.dt.is_quarter_start.astype('int8')
    df['is_quarter_end'] = df.date.dt.is_quarter_end.astype('int8')
    df['is_year_start'] = df.date.dt.is_year_start.astype('int8')
    df['is_year_end'] = df.date.dt.is_year_end.astype('int8')
    df['season'] = np.where(df.month.isin([12,1,2]), 0,
                             np.where(df.month.isin([6,7,8]), 2,
                                      np.where(df.month.isin([9,10,11]), 3, 1))).astype('int8')
    return df


def one_hot_encoder(df, nan_as_category=True):
    original = list(df.columns)
    cats = df.select_dtypes(['object','category']).columns.tolist()
    df_enc = pd.get_dummies(df, columns=cats, dummy_na=nan_as_category)
    df_enc.columns = df_enc.columns.str.replace(' ', '_')
    new_cols = [c for c in df_enc.columns if c not in original]
    return df_enc, new_cols


def create_oil_features(df, oil):
    """
    Create oil-related features from the oil price data
    
    Parameters:
    -----------
    df : pandas.DataFrame
        Main dataframe
    oil : pandas.DataFrame
        Oil price dataframe with 'date' and 'dcoilwtico' columns
        
    Returns:
    --------
    df : pandas.DataFrame
        Dataframe with added oil features
    """
    # Parse date if not already done
    if not pd.api.types.is_datetime64_dtype(oil['date']):
        oil['date'] = pd.to_datetime(oil['date'])
        
    # Interpolate oil price
    oil_series = (
        oil.set_index('date')['dcoilwtico']
           .replace(0, np.nan)
           .interpolate()
           .fillna(method='bfill')
           .rename('dcoilwtico_interpolated')
           .reset_index()
    )
    
    # Merge with main dataframe
    df = df.merge(oil_series, on='date', how='left')
    
    # Create binary feature for high oil price
    df['oil_above_70'] = (df['dcoilwtico_interpolated'] >= 70).astype('int8')
    
    # Drop intermediate column if not needed
    df.drop('dcoilwtico_interpolated', axis=1, inplace=True)
    
    return df


def create_holiday_features(df, holidays):
    """
    Create holiday-related features from the holidays data
    
    Parameters:
    -----------
    df : pandas.DataFrame
        Main dataframe
    holidays : pandas.DataFrame
        Holidays dataframe with required columns
        
    Returns:
    --------
    df : pandas.DataFrame
        Dataframe with added holiday features and work_day dataframe
    """
    # Parse date if not already done
    holidays["date"] = pd.to_datetime(holidays.date)

    
    # Ensure description is string type
    holidays['description'] = holidays['description'].fillna('').astype(str)
    
    # Fix "transferred" column if it's not a boolean
    if not pd.api.types.is_bool_dtype(holidays['transferred']):
        holidays['transferred'] = holidays['transferred'].astype(bool)
    
    # Transferred Holidays
    tr1 = holidays[(holidays.type == "Holiday") & (holidays.transferred == True)].drop("transferred", axis=1).reset_index(drop=True)
    tr2 = holidays[(holidays.type == "Transfer")].drop("transferred", axis=1).reset_index(drop=True)
    
    # Check if there are any rows before trying to concatenate
    if len(tr1) > 0 and len(tr2) > 0:
        tr = pd.concat([tr1, tr2], axis=1)
        tr = tr.iloc[:, [5,1,2,3,4]] if tr.shape[1] > 5 else tr  # Ensure proper indexing
    else:
        # Create an empty DataFrame with the same columns
        tr = pd.DataFrame(columns=holidays.columns)
    
    holidays = holidays[(holidays.transferred == False) & (holidays.type != "Transfer")].drop("transferred", axis=1)
    
    # Only concatenate if tr is not empty
    if not tr.empty:
        holidays = pd.concat([holidays, tr], axis=0).reset_index(drop=True)

    # Additional Holidays - safely apply string operations
    holidays["description"] = holidays["description"].str.replace("-", "", regex=False).str.replace("+", "", regex=False).str.replace(r"\d+", "", regex=True)
    holidays["type"] = np.where(holidays["type"] == "Additional", "Holiday", holidays["type"])

    # Bridge Holidays
    holidays["description"] = holidays["description"].str.replace("Puente ", "", regex=False)
    holidays["type"] = np.where(holidays["type"] == "Bridge", "Holiday", holidays["type"])

    # Work Day Holidays, that is meant to payback the Bridge
    work_day = holidays[holidays.type == "Work Day"]
    holidays = holidays[holidays.type != "Work Day"]

    # Split
    # Events are national
    events = holidays[holidays.type == "Event"].drop(["type", "locale", "locale_name"], axis=1).rename({"description":"events"}, axis=1)

    holidays = holidays[holidays.type != "Event"].drop("type", axis=1)
    
    # Only proceed if there are rows in holidays
    if not holidays.empty:
        regional = holidays[holidays.locale == "Regional"].rename({"locale_name":"state", "description":"holiday_regional"}, axis=1).drop("locale", axis=1).drop_duplicates()
        national = holidays[holidays.locale == "National"].rename({"description":"holiday_national"}, axis=1).drop(["locale", "locale_name"], axis=1).drop_duplicates()
        local = holidays[holidays.locale == "Local"].rename({"description":"holiday_local", "locale_name":"city"}, axis=1).drop("locale", axis=1).drop_duplicates()
    else:
        # Create empty DataFrames with the correct columns
        regional = pd.DataFrame(columns=["date", "state", "holiday_regional"])
        national = pd.DataFrame(columns=["date", "holiday_national"])
        local = pd.DataFrame(columns=["date", "city", "holiday_local"])

    # Merge National Holidays
    df = df.merge(national, how="left", on="date")
    
    # Regional - ensure 'state' column exists in both DataFrames
    if 'state' in df.columns:
        df = df.merge(regional, how="left", on=["date", "state"])
    else:
        # Add empty holiday_regional column if state column doesn't exist
        df['holiday_regional'] = np.nan
        
    # Local - ensure 'city' column exists in both DataFrames
    if 'city' in df.columns:
        df = df.merge(local, how="left", on=["date", "city"])
    else:
        # Add empty holiday_local column if city column doesn't exist
        df['holiday_local'] = np.nan

    # Work Day
    if not work_day.empty:
        df = df.merge(work_day[["date", "type"]].rename({"type":"IsWorkDay"}, axis=1), how="left")
    else:
        df["IsWorkDay"] = np.nan
    
    # EVENTS
    # Ensure events column is string type before using str methods
    if 'events' in events.columns and not events.empty:
        events['events'] = events['events'].fillna('').astype(str)
        events["events"] = np.where(events.events.str.contains("futbol", na=False), "Futbol", events.events)
        
        # One-hot encode events
        events_enc, events_cat = one_hot_encoder(events)
        
        # Special case for Mother's Day
        if 'events_Dia_de_la_Madre' in events_enc.columns and len(events_enc) > 239:
            mother_day_date = pd.to_datetime("2016-05-08")
            events_enc.loc[events_enc.date == mother_day_date, 'events_Dia_de_la_Madre'] = 1
            if 239 < len(events_enc):
                events_enc = events_enc.drop(239)
        
        df = df.merge(events_enc, how="left", on="date")
        if events_cat:  # Only try to fill if the list is not empty
            df[events_cat] = df[events_cat].fillna(0)
    
    # New features - safely handle possibly missing columns
    if 'holiday_national' in df.columns:
        df["holiday_national_binary"] = np.where(df.holiday_national.notna(), 1, 0).astype('int8')
    else:
        df["holiday_national_binary"] = 0
        
    if 'holiday_local' in df.columns:
        df["holiday_local_binary"] = np.where(df.holiday_local.notna(), 1, 0).astype('int8')
    else:
        df["holiday_local_binary"] = 0
        
    if 'holiday_regional' in df.columns:
        df["holiday_regional_binary"] = np.where(df.holiday_regional.notna(), 1, 0).astype('int8')
    else:
        df["holiday_regional_binary"] = 0
    
    # Additional holiday features
    if 'holiday_national' in df.columns:
        df["national_independence"] = np.where(
            df.holiday_national.isin([
                'Batalla de Pichincha', 'Independencia de Cuenca', 
                'Independencia de Guayaquil', 'Primer Grito de Independencia'
            ]), 
            1, 0
        ).astype('int8')
    else:
        df["national_independence"] = 0
    
    # Process local holiday features if they exist
    if 'holiday_local' in df.columns:
        # First ensure column is string type and fill NaN values
        df['holiday_local'] = df['holiday_local'].fillna('').astype(str)
        
        # Now safely apply string methods
        df["local_cantonizacio"] = np.where(df.holiday_local.str.contains("Cantonizacio", na=False), 1, 0).astype('int8')
        df["local_fundacion"] = np.where(df.holiday_local.str.contains("Fundacion", na=False), 1, 0).astype('int8')
        df["local_independencia"] = np.where(df.holiday_local.str.contains("Independencia", na=False), 1, 0).astype('int8')
    else:
        df["local_cantonizacio"] = 0
        df["local_fundacion"] = 0
        df["local_independencia"] = 0
    
    # One-hot encode holiday columns if they exist
    holiday_cols = ["holiday_national", "holiday_regional", "holiday_local"]
    existing_holiday_cols = [col for col in holiday_cols if col in df.columns]
    
    if existing_holiday_cols:
        for col in existing_holiday_cols:
            # Ensure values are strings before one-hot encoding
            df[col] = df[col].fillna('').astype(str)
            
        # Now it's safe to one-hot encode
        holidays_enc, holidays_cat = one_hot_encoder(df[existing_holiday_cols], nan_as_category=False)
        df = pd.concat([df.drop(existing_holiday_cols, axis=1), holidays_enc], axis=1)
    
    # Convert holiday columns to int8
    he_cols = (
        df.columns[df.columns.str.startswith("events")].tolist() + 
        df.columns[df.columns.str.startswith("holiday")].tolist() + 
        df.columns[df.columns.str.startswith("national")].tolist() + 
        df.columns[df.columns.str.startswith("local")].tolist()
    )
    
    # Only convert columns that exist and contain numeric data
    existing_he_cols = [col for col in he_cols if col in df.columns]
    if existing_he_cols:
        for col in existing_he_cols:
            try:
                df[col] = df[col].fillna(0).astype("int8")
            except (ValueError, TypeError):
                # If conversion fails, leave as is
                pass
    
    return df, work_day


# --- Core preprocessing pipeline ---

def preprocess_full(
    raw: pd.DataFrame,
    stores: pd.DataFrame,
    transactions: pd.DataFrame,
    oil: pd.DataFrame,
    holidays: pd.DataFrame
) -> pd.DataFrame:
    # parse dates
    raw['date'] = pd.to_datetime(raw['date'])
    
    # merge store info
    df = raw.merge(stores, on='store_nbr', how='left')

    df, work_day = create_holiday_features(df, holidays)
    df = create_date_features(df)

    # add oil features
    df = create_oil_features(df, oil)
    
    # date features first (needed for workday calculation)
    
    # add holiday features

    # flags
    df['workday'] = (~(
        (df.holiday_national_binary==1) |
        (df.holiday_regional_binary==1) |
        (df.holiday_local_binary==1) |
        (df.day_of_week.isin([6,7]))
    )).astype('int8')
    
    # Safely handle work_day
    if not work_day.empty and 'date' in work_day.columns:
        df.loc[df.date.isin(work_day.date), 'workday'] = 1
        
    df['wageday'] = ((df.is_month_end==1)|(df.day_of_month==15)).astype('int8')

    # categories
    for c in ['family','city','state','type','cluster','oil_above_70']:
        if c in df.columns:  # Only try to convert if the column exists
            df[c] = df[c].astype('category')

    return df

# --- Feature inspection interface ---

def inspect_features_from_dicts(
    input_data: dict,
    stores_data: dict,
    transactions_data: dict,
    oil_data: dict,
    holidays_data: dict
) -> pd.DataFrame:
    norm = {}
    for k, v in input_data.items():
        norm[k] = v if isinstance(v, (list, np.ndarray)) else [v]
    df_raw = pd.DataFrame(norm)

    stores = pd.DataFrame(stores_data)
    transactions = pd.DataFrame(transactions_data)
    oil = pd.DataFrame(oil_data)
    holidays = pd.DataFrame(holidays_data)

    df_feat = preprocess_full(df_raw, stores, transactions, oil, holidays)

    print("=== Engineered Feature Sample ===")
    display(df_feat.head())
    return df_feat

# if __name__ == '__main__':
#     example_input = {
#         'id': 3000888,
#         'date': '2017-08-16',
#         'store_nbr': 1,
#         'family': 'AUTOMOTIVE',
#         'onpromotion': 0
#     }
#     stores_dict = pd.read_csv('stores.csv')
#     transactions_dict = pd.read_csv('transactions.csv')
#     oil_dict = pd.read_csv('oil.csv')
#     holidays_dict = pd.read_csv('holidays_events.csv')

#     _ = inspect_features_from_dicts(
#         example_input,
#         stores_dict,
#         transactions_dict,
#         oil_dict,
#         holidays_dict
#     )
if __name__ == '__main__':
    # 1) Load all CSVs
    raw       = pd.read_csv('train.csv',  parse_dates=['date'])
    stores    = pd.read_csv('stores.csv')
    transactions = pd.read_csv('transactions.csv', parse_dates=['date'])
    oil       = pd.read_csv('oil.csv',      parse_dates=['date'])
    holidays  = pd.read_csv('holidays_events.csv')

    # 2) Preprocess the entire dataset
    df = preprocess_full(
        raw=raw,
        stores=stores,
        transactions=transactions,
        oil=oil,
        holidays=holidays
    )

    # 3) (Optional) Save to disk for later training
    df.to_csv('train_preprocessed.csv', index=False)
    print(f'Preprocessed data shape: {df.shape}')



Preprocessed data shape: (3002670, 84)


In [55]:
len(df.columns)


84

In [56]:
import json

# 4) Save the list of all columns seen in training
feature_cols = df.columns.tolist()
with open('feature_columns.json', 'w') as f:
    json.dump(feature_cols, f)
